In [ ]:
import json
from qdrant_client import QdrantClient

# Buka file config.json dan baca API key
with open("config.json", "r") as file:
  config = json.load(file)

# Gunakan API key untuk koneksi ke Qdrant Cloud
client = QdrantClient(url=config["QDRANT_URL"], api_key=config["QDRANT_API_KEY"])

# Cek apakah koneksi berhasil
print(client.get_collections())